In [9]:
######################################################################
# Author: Alex Kim
# Date: 2024-04-26
#
# This script parses and converts encoded csv data into decoded SPSS
# file with variable labels derived from the encoded file.
#
# Usage:
# - Ensure the required libraries are installed in current R env
# - Update the 3 variables for input/output file paths
# - Run the script
######################################################################
suppressPackageStartupMessages({
    library(readr)
    library(dplyr)
    library(haven)
})
######################################################################
# User defined variables

data_dict_path <- './data/data_dictionary.csv'
encoded_data_path <- './data/data_encoded_04_24_2024.csv'
spss_output_path <- './data/spss-output_04_24_2024-v5.sav'

######################################################################

load_files <- function(encoded_data_path, data_dict_path) {
    data_dict <- read_csv(data_dict_path, show_col_types = FALSE)
    # Extract column names and descriptions from the first two rows before actually loading data
    tmp_data <- read_csv(encoded_data_path, col_names = FALSE, show_col_types = FALSE)
    col_names <- as.character(tmp_data[1, ])
    # Reload the data, skipping the first two rows to avoid the col description row
    data <- read_csv(encoded_data_path, skip = 2, col_names = col_names, show_col_types = FALSE)

    return(list(data = data, data_dict = data_dict))
}

# Important for adding variable labels for SPSS.
# The 2nd row in the encoded data files contain column descriptions.
get_col_descriptions <- function(encoded_data_path) {
    df <- read_csv(encoded_data_path, col_names = FALSE, show_col_types = FALSE)
    col_names <- as.character(df[1, ])
    col_descriptions <- as.character(df[2, ])
    col_descriptions <- setNames(col_descriptions, col_names)

    return(col_descriptions)
}

# Add variable labels as metadata for SPSS
add_var_labels <- function(data, encoded_data_path) {
    # Add variable labels for SPSS from the 2nd row of encoded data files
    col_descriptions <- get_col_descriptions(encoded_data_path)

    for(col in names(col_descriptions)) {
        if(col %in% names(data)) {
            attr(data[[col]], "label") <- col_descriptions[[col]]
        }
    }
    return (data)
}

# Parse the data dict for the encoded variable values
# and add as value labels for SPSS file.
get_value_labels <- function(data_dict) {
  # Filter out duplicate value-label pairs
  data_dict <- data_dict[!duplicated(paste(data_dict$variable, data_dict$label)), fromLast = TRUE]

   # Cast 'label' column to integers for nicer output in SPSS
  data_dict$label <- as.integer(data_dict$label)  
   
  # Transform the data dictionary into a list of value labels
  value_labels_list <- split(data_dict, data_dict$variable)
  
  # Transform into a named vector for each variable
  value_labels <- lapply(value_labels_list, function(x) {
    setNames(x$label, x$value)
  })
  
  return(value_labels)
}

# Function to add value labels to the data
add_value_labels <- function(data, data_dict_path) {
    value_labels <- get_value_labels(data_dict_path)
    for(var in names(value_labels)) {
        # Convert the list of labels to a named vector
        labels_vector <- value_labels[[var]]

        # Try casting to int here for pretty SPSS labels
        data[[var]] <- labelled(as.integer(data[[var]]), labels = labels_vector)    
    }
    return(data)
}

main <- function(encoded_dath_path, data_dict_path, spss_output_path) {
    # Import the encoded data and the data dict
    result <- load_files(encoded_data_path, data_dict_path)
    data <- result$data
    data_dict <- result$data_dict

    # Add factor/value labels for SPSS
    data <- add_value_labels(data, data_dict)
    
    # Add variable labels for SPSS
    data <- add_var_labels(data, encoded_data_path)
    
    # Export the processed data to an SPSS file
    write_sav(data, spss_output_path)
    print(paste("Data exported successfully to", spss_output_path))
    return (data)
}

df <- main(encoded_data_path, data_dict_path, spss_output_path)

[1] "Data exported successfully to ./data/spss-output_04_24_2024-v5.sav"


In [5]:
val_labels <- get_value_labels(data_dict)
val_labels

$demographics_age
other 
  -99 

$demographics_athlete
   none    club varsity   other 
      3       2       1       4 

$demographics_financialAid
   no   yes other 
    2     1     3 

$demographics_firstGen
          Attended college but did not complete degree 
                                                     3 
                            Did not finish high school 
                                                     1 
                         High school diploma or G.E.D. 
                                                     2 
Doctoral or professional degree (Ph.D. J.D. M.D. etc.) 
                                                     7 
                   Associate’s degree (A.A. A.S. etc.) 
                                                     4 
                    Bachelor’s degree (B.A. B.S. etc.) 
                                                     5 
                      Master’s degree (M.A. M.S. etc.) 
                                                     6 

$demographics_gender
             nonbinary                 female                   male 
                     3                      2                      1 
                unsure                agender genderQueerGenderFluid 
                     6                      4                      5 

$demographics_highschool
public charter         public        private  public magnet      parochial 
             2              1              5              3              4 
         other     homeschool 
             7              6 

$demographics_major_one
                Agricultural/Food Sciences 
                                         3 
                               Visual Arts 
                                        72 
                          Computer Science 
                                        19 
          Psychological and Brain Sciences 
                                        57 
                              Neuroscience 
                                        50 
                              Architecture 
                                         7 
        Biochemistry and Molecular Biology 
                                        12 
                                  Business 
                                        14 
                            Animal Science 
                                         4 
                              Anthropology 
                                         5 
                                 Economics 
                                        25 
              African/Afr-American Studies 
                                         2 
                       Biological Sciences 
                                        13 
                      Engineering Sciences 
                                        27 
                                Accounting 
                                         1 
                                 Chemistry 
                                        15 
                                Studio Art 
                                        68 
                           Law/Criminology 
                                        44 
                             Public Health 
                                        58 
                                 Marketing 
                                        46 
                         Cognitive Science 
                                        17 
                              Data Science 
                                        21 
                                   History 
                                        39 
                                     other 
                                        74 
                                   English 
                                        28 
                                   Nursing 
                                        51 
                   International Relations 
                                        41 
                               Art History 
                  

In [8]:
# Function to check if all columns have labels
check_col_labels <- function(data) {
    for(col in names(data)) {
        label <- attr(data[[col]], "label")
        if(is.null(label)) {
            print(paste("Column:", col, "- Missing label"))
        } else {
            # print(paste("Column:", col, "- Label:", label))
        }
    }
}

library(haven)

# Function to check value labels assigned to variables
check_value_labels <- function(data) {
    for(col in names(data)) {
        if(is.labelled(data[[col]])) {
            labels <- attr(data[[col]], "labels")
            if(!is.null(labels)) {
                print(paste("Column:", col, "- Value Labels:"))
                for (key in names(labels)) {
                    print(paste("  ", key, "-", labels[key]))
                }
            } else {
                print(paste("Column:", col, "- No Value Labels Assigned"))
            }
        } else {
            print(paste("Column:", col, "- Not a labelled variable"))
        }
    }
}



# check_col_labels(df)
check_value_labels(df)

[1] "Column: ResponseId - Not a labelled variable"
[1] "Column: RecordedDate - Not a labelled variable"
[1] "Column: institution_unitid - Not a labelled variable"
[1] "Column: institution_institutionName - Not a labelled variable"
[1] "Column: institution_totalEnrollment - Not a labelled variable"
[1] "Column: institution_undergraduateEnrollment - Not a labelled variable"
[1] "Column: institution_sectorOfInstitution - Not a labelled variable"
[1] "Column: institution_fipsStateCode - Not a labelled variable"
[1] "Column: demographics_age - Value Labels:"
[1] "   other - -99"
[1] "Column: demographics_athlete - Value Labels:"
[1] "   none - 3"
[1] "   club - 2"
[1] "   varsity - 1"
[1] "   other - 4"
[1] "Column: demographics_financialAid - Value Labels:"
[1] "   no - 2"
[1] "   yes - 1"
[1] "   other - 3"
[1] "Column: demographics_gender - Value Labels:"
[1] "   nonbinary - 3"
[1] "   female - 2"
[1] "   male - 1"
[1] "   unsure - 6"
[1] "   agender - 4"
[1] "   genderQueerGenderFluid -

In [3]:
result = load_files(encoded_data_path, data_dict_path)
data = result$data
data_dict = result$data_dict




In [ ]:
# Removing dups from data_dict

dim(data_dict)
data_dict <- data_dict[!duplicated(paste(data_dict$variable, data_dict$label)), fromLast = TRUE]
dim(data_dict)